## Loading up packages

In [1]:
import sys
sys.path.append('/home/keeganfl/Desktop/Work_Fall_2021/Fall_2021_Work_journal/tools/')

In [2]:
# import plastid
# data structure for mapping read alignments to genomic positions
from plastid import BAMGenomeArray, GenomeArray, VariableFivePrimeMapFactory, \
                        GTF2_TranscriptAssembler, GFF3_TranscriptAssembler, \
                        Transcript, ThreePrimeMapFactory, CenterMapFactory
import plastid
from plastid.plotting.plots import *
import numpy as np
import os
from Bio import SeqIO
import numpy
import math
import pandas as pd
from scipy import stats
import keegan_analysis_tools as kat
from statsmodels.nonparametric.smoothers_lowess import lowess
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from scipy.sparse.linalg import lsqr
from tqdm import tqdm

In [3]:
# Define the path to our Bam files
data_path = "/home/keeganfl/Desktop/Work_Fall_2021/genomes_&_samples/dmel/"
position_count_path = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/position_counts/dmel/"
te_table_path = '/home/keeganfl/Desktop/Work_Fall_2021/data_tables/TE_tables/dmel/'
save_path = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/elongation_rates/dmel/"
save_path2 = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/initiation_rates/dmel/"
save_path3 = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/termination_rates/dmel/"
gtf_name = "Drosophila_melanogaster.BDGP6.32.103.gtf"
mutation = 'control'
samp_num = '3'

## Loading up the data files 

In [4]:
# load the transcript annotations from the GTF file.
# GTF2_TranscriptAssembler returns an iterator, so here we convert it to a list.
transcripts = list(GTF2_TranscriptAssembler(open(data_path + gtf_name),return_type=Transcript))

In [5]:
# Remove non-protein coding transcripts from transcripts list. 
protein_coding = []
for transcript in transcripts:
    if transcript.attr['gene_biotype'] == 'protein_coding':
        protein_coding.append(transcript)

In [6]:
# clear up some memory by deleting original transcript list
transcripts.clear()

# Loading up the position counts file.

In [7]:
data = kat.load_count_positions(mutation + "_RPF_" + samp_num + "_counts.csv",position_count_path)

## Loading up the TE table

In [8]:
te_table = pd.read_csv(te_table_path + mutation + "_" + samp_num + "_te_tablefc.csv")

## Getting the codon sequence:

If get.sequence returns the entire sequence, then it should be fairly simple to just index it with the numbers returned by get_cds stop and end so that I only have to look at the coding regions... Now how am I going to read through this such that each codon gets its own little name? 

Perhaps I should consider trying to run biopython in such a way that I do not have to save it into it's own dictionary like this.

In [9]:
genome = SeqIO.to_dict(SeqIO.parse(
    open(data_path + "Drosophila_melanogaster.BDGP6.32.cds.all.fa"),"fasta"))
# Using this cds file leads to everything being in 

In [10]:
cds_sequence=[]

for transcript in protein_coding:
    t_id = transcript.attr["transcript_id"]
    cds_sequence.append(str(genome[t_id].seq))

In [30]:
cds_sequence[1]

'ATGTCGAGCTCCCAGAACAACAACAGCAGCTGGATCGATTGGTTCCTCGGGATCAAGGGCAACGAGTTCCTCTGCCGCGTGCCCACCGACTACGTGCAGGATACGTTCAACCAGATGGGCTTGGAGTATTTCAGCGAGATACTGGACGTGATCCTGAAGCCGGTGATCGACAGTTCCTCTGGCTTGTTGTACGGCGATGAAAAAAAGTGGTACGGCATGATTCACGCCCGATACATCAAGTCAGAGCGTGGCGTGATTGCTATGCACCGAAAATATATGCGAGGAGATTTTGAATCGTGTCCCAATATCTCCTGTGATAGGCAGAACACCCTGCCAGTGGGCCTCAGCGATGTATGGGGCAAGTCAACCGTCAAGATCTACTGCCCACGGTGTAAAAAGAACTTTCATCCGAAGTCTGATACACAGCTGGACGGAGCGATGTTCGGGCCCAGCTTCCCGGACATCTTCTTCTCGCTGCTGCCGAACTTGAGATCGCCCCTGGACGACCCACGTACCTAG'

In [32]:
codon_seq_list[1]

[50,
 6,
 57,
 5,
 30,
 61,
 61,
 61,
 57,
 57,
 10,
 49,
 44,
 10,
 1,
 17,
 42,
 49,
 62,
 41,
 61,
 46,
 1,
 17,
 9,
 25,
 34,
 21,
 53,
 45,
 13,
 34,
 30,
 44,
 54,
 1,
 61,
 30,
 50,
 41,
 2,
 46,
 12,
 1,
 57,
 46,
 51,
 18,
 45,
 34,
 49,
 18,
 62,
 22,
 34,
 49,
 45,
 56,
 5,
 4,
 41,
 2,
 2,
 13,
 41,
 44,
 47,
 63,
 62,
 10,
 13,
 41,
 50,
 48,
 29,
 37,
 27,
 13,
 49,
 62,
 7,
 46,
 24,
 41,
 34,
 48,
 36,
 50,
 29,
 27,
 63,
 12,
 50,
 27,
 43,
 44,
 0,
 47,
 6,
 8,
 21,
 60,
 49,
 5,
 8,
 44,
 58,
 30,
 61,
 53,
 18,
 23,
 34,
 41,
 17,
 57,
 44,
 35,
 10,
 41,
 62,
 7,
 53,
 33,
 62,
 49,
 13,
 9,
 23,
 26,
 8,
 63,
 62,
 61,
 0,
 28,
 22,
 62,
 4,
 44,
 55,
 30,
 18,
 45,
 43,
 38,
 50,
 1,
 42,
 21,
 57,
 1,
 22,
 45,
 49,
 1,
 1,
 6,
 18,
 18,
 22,
 61,
 2,
 59,
 6,
 21,
 18,
 45,
 45,
 23,
 24,
 53,
 14]

In [11]:
cd_dict = {'TTT':0,'TTC':1,'TTG':2,'TTA':3,
          'TCT':4,'TCC':5,'TCG':6,'TCA':7,
          'TGT':8,'TGC':9,'TGG':10,'TGA':11,
          'TAT':12,'TAC':13,'TAG':14,'TAA':15,
          'CTT':16,'CTC':17,'CTG':18,'CTA':19,
          'CCT':20,'CCC':21,'CCG':22,'CCA':23,
          'CGT':24,'CGC':25,'CGG':26,'CGA':27,
          'CAT':28,'CAC':29,'CAG':30,'CAA':31,
          'GTT':32,'GTC':33,'GTG':34,'GTA':35,
          'GCT':36,'GCC':37,'GCG':38,'GCA':39,
          'GGT':40,'GGC':41,'GGG':42,'GGA':43,
          'GAT':44,'GAC':45,'GAG':46,'GAA':47,
          'ATT':48,'ATC':49,'ATG':50,'ATA':51,
          'ACT':52,'ACC':53,'ACG':54,'ACA':55,
          'AGT':56,'AGC':57,'AGG':58,'AGA':59,
          'AAT':60,'AAC':61,'AAG':62,'AAA':63}

In [12]:
codon_seq_list = []

for seq in cds_sequence:
    codon_seq = []
    for i in range(0, len(seq), 3):
        try:
            codon_seq.append(cd_dict[seq[i:i+3]])
        except:
            pass
    codon_seq_list.append(codon_seq)

## Calculating Elongation rates

In [13]:
# Create a function to obtain a normalized profile (p) of ribosome footprints.
def calculate_p(data):
    p_list=[]
    for i in data:
        i = i+1
        M = sum(i)
        p = i/M
        p_list.append(p)
    return(p_list)

def calculate_p_chi(data_counts, data_te, chi = 0.00289):
    p_list = []
    for i, j in zip(data_counts, te_table.TE_rpkm):
        pi = j*chi
        i = i+1
        M = sum(i)
        p = (i/M)*float(pi)*(len(i)-1)
        p_list.append(p)
    return(p_list)

In [14]:
p_list = calculate_p_chi(data, te_table, 0.00289)

In [15]:
pbar_list = kat.calculate_pbar(p_list)

In [16]:
lbar_list = kat.calculate_lbar(pbar_list)

---------------------------------------------------------------------------
RuntimeWarning
invalid value encountered in double_scalars
in /home/keeganfl/Desktop/Work_Fall_2021/Fall_2021_Work_journal/tools/keegan_analysis_tools.py, line 411:

409                 lbar_x=9999
410             else:
411                 lbar_x = (1-9*pbarx)/(pbarx*(1-pbarx))
412             lbar.append(lbar_x)
413         lbar_list.append(np.array(lbar))

---------------------------------------------------------------------------


In [17]:
# calculate the scaled initiation and termination rates

init_r= []
for pbar in pbar_list:
    if pbar[0] == 0:
        init_r.append(1/(1-10*0.00001))
    else:
        init_r.append(1/(1-10*pbar[0]))

term_r = []
for p in p_list:
    if p[-1] ==0:
        term_r.append(1/0.00001)
    else:
        term_r.append(1/(p[-1]))


In [18]:
tau_list = kat.calculate_tau(lbar_list, codon_seq_list)

30710it [05:33, 92.06it/s] 


In [19]:
elongation_list = np.array(lbar_list, dtype=object)/np.array(tau_list, dtype=object)

In [20]:
elongation_list = elongation_list.tolist()

In [21]:
for elon, i in zip(elongation_list, list(range(len(elongation_list)))):
    elongation_list[i] = elon.tolist()

In [22]:
initiation_list = np.array(init_r, dtype=object)/np.array(tau_list, dtype =object)

In [23]:
np.savetxt(save_path2 + mutation + '_' + samp_num + '_adjusted_initiation_rates', initiation_list, delimiter = ",")

In [24]:
termination_list = np.array(term_r, dtype=object)/np.array(tau_list, dtype =object)

In [25]:
np.savetxt(save_path3 + mutation + '_' + samp_num + '_adjusted_termination_rates', termination_list, delimiter = ",")

## Save as a csv

In [26]:
# Create a list that contains all of the gene_ids and transcript_ids of the transcripts
gene_id = []
transcript_id = []

for transcript in protein_coding:
    gene_id.append(transcript.attr["gene_name"])
    transcript_id.append(transcript.attr["transcript_id"])

In [27]:
# Insert the gene ids and transcript ids into the codon_count list. 
for i,j in zip(elongation_list, range(len(gene_id))):
    i.insert(0,gene_id[j])
    i.insert(0,transcript_id[j])

In [28]:
# Calculate the longest cds region in our new list of counts
l_tr = kat.find_max_list(elongation_list)

# Define a header that includes labels for the transcript and gene ID as 
# well as numbers that index the cds region position.
header=["transcript_id","gene_id"]+list(range(l_tr))

# insert that header into our counts list. 
elongation_list.insert(0,header)

In [29]:
with open(save_path + mutation + '_' + samp_num + '_adjusted_elongation_rates', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(elongation_list)